In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Emotion_final.csv")
# data

In [3]:
data['Emotion'].value_counts()

happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: Emotion, dtype: int64

In [4]:
def clear_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^ a-z]', '', text)
    while text.find('  ') != -1:
        text = text.replace('  ', ' ')
    return text

data['clean_text']= data['Text'].apply(lambda x:clear_text(x))

In [5]:
data['tokenied'] = data['clean_text'].apply(lambda x:word_tokenize(x))

In [6]:
stop_words = stopwords.words('english')

def remove_stop_words(text):
    filtered_text = []
    for w in text:
        if w not in stop_words:
            filtered_text.append(w)
    return filtered_text

data['no_stopwords'] = data['tokenied'].apply(lambda x:remove_stop_words(x))

In [7]:
# lemmatizer = WordNetLemmatizer()

wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    return [wordnet_lemmatizer.lemmatize(w) for w in text]

data['to_train'] = data['no_stopwords'].apply(lambda x:lemmatizer(x))
# data[['no_stopwords', 'lemmatized']]

In [8]:
text = ''
train_text = data.Text
for i in train_text:
    text += i + ' '
text = clear_text(text)

In [9]:
tokens = word_tokenize(text)
tokens = remove_stop_words(tokens)
lemmatizer = WordNetLemmatizer()
lemtok = [lemmatizer.lemmatize(t) for t in tokens]
word_list = list(lemtok)
word_list.sort()
# word_list

In [10]:
dictionary = []
for i in word_list:
    if i not in dictionary:
        dictionary.append(i)
dictionary

['aa',
 'aaaaaaand',
 'aaaaand',
 'aaaah',
 'aaaand',
 'aac',
 'aahhh',
 'aaron',
 'ab',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abated',
 'abba',
 'abbigail',
 'abc',
 'abd',
 'abdomen',
 'abdominal',
 'abducted',
 'abelard',
 'abhorrent',
 'abide',
 'abigail',
 'ability',
 'abit',
 'abjectly',
 'able',
 'ableness',
 'ablo',
 'abnormally',
 'aboard',
 'abominable',
 'abortion',
 'abou',
 'abound',
 'abraham',
 'abroad',
 'abruptly',
 'absence',
 'absoloutely',
 'absolute',
 'absolutely',
 'absolutly',
 'absorbed',
 'absorption',
 'abstain',
 'abstinence',
 'abstract',
 'absurd',
 'absurdity',
 'absurdly',
 'abt',
 'abu',
 'abundance',
 'abundantly',
 'abus',
 'abuse',
 'abused',
 'abusing',
 'abusive',
 'abyss',
 'ac',
 'academia',
 'academic',
 'academy',
 'acause',
 'accelerated',
 'accent',
 'accentuating',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accessary',
 'accessibility',
 'accessible',
 'accessory',
 'acci

In [11]:
def convert_to_numbers(text):
    res = []
    for word in text:
        res.append(dictionary.index(word))
    return res

def convert_to_text(numbers):
    res = []
    for num in numbers:
        res.append(dictionary[num])
    return res

In [ ]:
data['numbers'] = data['to_train'].apply(lambda x:convert_to_numbers(x))
# data['numbers']

In [ ]:
# data['text2'] = data['numbers'].apply(lambda x:convert_to_text(x))
# data['text2']

In [ ]:
#good: happy, love, surprise
#bad: sadness, anger, fear

def t_or_f(emotion):
    if emotion == 'happy' or emotion == 'love' or emotion == 'surprise':
        return 1        
    else:        #elif emotioin == 'sadness' or 'anger' or 'fear': return 0
        return 0
    
data['Emotion_in_digit'] = data['Emotion'].apply(lambda x:t_or_f(x))
# data[['Emotion','Emotion_in_digit']]

In [ ]:
train_data = data[['Emotion_in_digit', 'numbers']]
train_data

In [ ]:
def vectorize_sequences(sequences, dimension=30000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

t = vectorize_sequences(train_data['numbers'])

